In [2]:
import numpy as np
import xarray as xr

In [65]:
file_in = "../latlon_iso_for_VP.nc"
time_file_in = "../fixed_latlon_diag_sl.nc"
diag_file = "../diag.2021-09-24_00.00.00.nc"

In [72]:
out_file_general = "mpas_data_multiple_levels.nc"
out_file_lat = "mpas_data_flipped_lat.nc"
out_file_lat_lon_name = "mpas_data_name_lat.nc"
out_file_lat_lon = "mpas_data_flipped_lat_lon.nc"
out_file_extra_variable = "mpas_data_extra_data.nc"
out_file_missing_data = "mpas_data_missing_data.nc"
out_file_mislabelled_data = "mpas_data_mislabel_data.nc"
out_file_mislabelled_lat = "mpas_data_mislabel_lat.nc"

In [73]:
data_xr = xr.open_dataset(file_in)
time_xr = xr.open_dataset(time_file_in)["time"]
diag_xr = xr.open_dataset(diag_file)
data_xr = data_xr.rename_dims({"Time": "time"})
data_xr["time"] = time_xr
data_xr.coords["u_iso_levels"] = diag_xr["u_iso_levels"]
data_xr = data_xr.rename({"uzonal_isobaric": "u", "umeridional_isobaric": "v", "u_iso_levels": "level"})

In [74]:
data_xr["longitude"] = np.round(data_xr.longitude)
data_xr["latitude"] = np.round(data_xr.latitude)
data_xr = data_xr.reindex(latitude=list(reversed(data_xr.latitude)))

In [75]:
data_xr.to_netcdf(out_file_general)

In [76]:
# data_xr.to_netcdf(out_file_general)
data_lat_flipped = data_xr.reindex(latitude=list(reversed(data_xr.latitude)))
data_lat_lon_flipped = data_lat_flipped.reindex(longitude=list(reversed(data_lat_flipped.longitude)))
data_lat_lon_name_xr = data_xr.rename({"latitude": "lat", "longitude": "lon"})
data_extra_var_xr = xr.merge([data_xr, diag_xr])
data_missing_data_xr = data_xr.sel(latitude=60).squeeze()
data_mislabelled_data = data_xr.rename({"u": "u_wind"})
data_mislabelled_lat = data_xr.rename_dims({"latitude": "lats"})

In [80]:
data_lat_flipped.to_netcdf(out_file_lat)
data_lat_lon_flipped.to_netcdf(out_file_lat_lon)
data_lat_lon_name_xr.to_netcdf(out_file_lat_lon_name)
data_extra_var_xr.to_netcdf(out_file_extra_variable)
data_missing_data_xr.to_netcdf(out_file_missing_data)
data_mislabelled_data.to_netcdf(out_file_mislabelled_data)
data_mislabelled_lat.to_netcdf(out_file_mislabelled_lat)

In [64]:
data_mislabelled_data

<xarray.Dataset> Size: 15MB
Dimensions:    (longitude: 360, time: 193, levels: 27)
Coordinates:
    latitude   float32 4B 60.0
  * longitude  (longitude) float32 1kB -180.0 -179.0 -178.0 ... 178.0 179.0
  * time       (time) datetime64[ns] 2kB 2021-09-24 ... 2021-10-02
  * levels     (levels) float32 108B 1e+04 1.25e+04 1.5e+04 ... 9.75e+04 1e+05
Data variables:
    v          (time, levels, longitude) float32 8MB ...
    u          (time, levels, longitude) float32 8MB ...